In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib as plt

In [2]:
from NRI.experiments.config import ExperimentConfig


experiment_config = ExperimentConfig(
    obs_len=12,
    pred_len=12,
    encoder_loss_weight=1.0,
    decoder_loss_weight=1.0,
    signal_loss_weight=1.0,
    n_epoch=20,
    batch_size=64,
    checkpoint_prefix='dnri_12_baseline',
    checkpoint_interval=10,
    use_cuda=True,
    mask_traffic_signal=False
)

In [3]:
from SinD.config import get_dataset_path
from SinD.dataset.io import get_dataset_records
from NRI.dataset.utils import split_dataset

dataset_path = get_dataset_path()
dataset_files = get_dataset_records(dataset_path)
train_files, valid_files, test_files = split_dataset(dataset_files, 0.7, 0.2)

load datasets

In [4]:
from NRI.dataset import SignalizedIntersectionDatasetForNRI, SignalizedIntersectionDatasetConfig

dataset_config = SignalizedIntersectionDatasetConfig(
    obs_len=experiment_config.obs_len,
    pred_len=experiment_config.pred_len,
    stride=15,
    encode_traffic_signals=True,
    padding_value=0.0
)

train_set = SignalizedIntersectionDatasetForNRI(dataset_config)
train_set.load_records(dataset_path, train_files, verbose=True)

valid_set = SignalizedIntersectionDatasetForNRI(dataset_config)
valid_set.load_records(dataset_path, valid_files, verbose=True)


load_records: 100%|██████████| 4/4 [00:35<00:00,  8.86s/it]


In [5]:
from NRI.models import DynamicNeuralRelationalInference
from NRI.experiments.main import train

model = DynamicNeuralRelationalInference(
    hid_dim=64,
    n_edges=4,
    dgvae=False
)

if experiment_config.use_cuda:
    model.cuda()

optimizer = torch.optim.Adam(model.parameters())

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

train(
    model,
    train_set=train_set,
    valid_set=valid_set,
    config=experiment_config,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler
)

[train] epoch 0:   0%|          | 0/174 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

Overall Loss: 195.6832767629074
Encoder KL Loss: 0.04281331140055566
Decoder NLL Loss: 194.59245202733186
Signal Cross-Entropy Loss: 1.048010437652983


[valid] epoch 0:   0%|          | 0/49 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: Th

Signal Prediction Accuracy: 0.7915444374084473
Final Displacement Error: 6.26243545571152
Average Displacement Error: 3.0850474737128435


[train] epoch 1:   0%|          | 0/174 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

...

Signal Prediction Accuracy: 0.827949583530426
Final Displacement Error: 4.753188829032743
Average Displacement Error: 2.2289942746259728


[train] epoch 19:   0%|          | 0/174 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: 

Overall Loss: -6.099895799982141
Encoder KL Loss: 0.06852024084963328
Decoder NLL Loss: -6.729240174951221
Signal Cross-Entropy Loss: 0.5608241475861646


[valid] epoch 19:   0%|          | 0/49 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

Signal Prediction Accuracy: 0.824415385723114
Final Displacement Error: 4.724471379299553
Average Displacement Error: 2.2147310436988366


Execute test set

In [1]:
import os
import torch
from torch.utils.data import DataLoader

from NRI.dataset import SignalizedIntersectionDatasetForNRI
from NRI.models import DynamicNeuralRelationalInference
from NRI.experiments.config import ExperimentConfig
from NRI.experiments.test import generate_result
from SinD.config import get_dataset_path
from SinD.dataset.io import get_dataset_records
from NRI.dataset.utils import split_dataset
from NRI.dataset import SignalizedIntersectionDatasetForNRI, SignalizedIntersectionDatasetConfig

experiment_config = ExperimentConfig(
    obs_len=12,
    pred_len=12,
    encoder_loss_weight=1.0,
    decoder_loss_weight=1.0,
    signal_loss_weight=1.0,
    n_epoch=20,
    batch_size=64,
    checkpoint_prefix='dnri_12_baseline',
    checkpoint_interval=10,
    use_cuda=True,
    mask_traffic_signal=False
)

# Load the dataset
dataset_path = get_dataset_path()
dataset_files = get_dataset_records(dataset_path)
_, _, test_files = split_dataset(dataset_files, 0.7, 0.2)

dataset_config = SignalizedIntersectionDatasetConfig(
    obs_len=experiment_config.obs_len,
    pred_len=experiment_config.pred_len,
    stride=15,
    encode_traffic_signals=True,
    padding_value=0.0,
    include_incomplete_trajectories=False
)

test_set = SignalizedIntersectionDatasetForNRI(dataset_config)
test_set.load_records(dataset_path, test_files, verbose=True)

model = DynamicNeuralRelationalInference(
    hid_dim=64,
    n_edges=4,
    dgvae=False
)


# Load the saved model and configuration
checkpoint = torch.load('../checkpoints/dnri_12_baseline_best.pt')
model.load_state_dict(checkpoint['params'])
test_stats = generate_result(model, test_set, experiment_config)

# Report the results
test_stats.report()

[test] generating result:   0%|          | 0/34 [00:00<?, ?it/s]c:\Users\chinp\source\AnythingVisionaries\venv\Lib\site-packages\torch\nested\__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ..\aten\src\ATen\NestedTensorImpl.cpp:180.)
  return _nested.nested_tensor(
[test] generating result: 100%|██████████| 34/34 [00:06<00:00,  5.66it/s]

=== Overall Result ===
Signal Prediction Accuracy: 0.7718290090560913
Overall Final Displacement Error: 1.9836796522140503
Overall Average Displacement Error: 0.7808113098144531
=== Displacement Error of Class `pedestrian` ===
FDE: 3.556249748331029
ADE: 1.9006368871441541
=== Displacement Error of Class `car` ===
FDE: 2.210488477220763
ADE: 0.5796978505316064
=== Displacement Error of Class `truck` ===
FDE: 20.928002699728935
ADE: 5.911805153854432
=== Displacement Error of Class `bus` ===
FDE: 0.05091768265484478
ADE: 0.02343659750122238
=== Displacement Error of Class `motorcycle` ===
FDE: 2.2600996758829237
ADE: 0.8208592312035741
=== Displacement Error of Class `tricycle` ===
FDE: 1.1623582486798372
ADE: 0.4981800039675467
=== Displacement Error of Class `bicycle` ===
FDE: 3.5434410093189834
ADE: 1.521978985106761
=== Overall Result ===
Signal Prediction Accuracy: 0.7718290090560913
Overall Final Displacement Error: 1.9836796522140503
Overall Average Displacement Error: 0.78081130